In [9]:
import pyspark
SPARK_VERSION = '3.1.2'
SCALA_VERSION = '2.12'

### adiciona package adicional
import findspark
findspark.add_packages(['org.apache.spark:spark-sql-kafka-0-10_' + SCALA_VERSION + ':' + SPARK_VERSION ])
findspark.init()

import pandas as pd
import pyarrow

import mysql.connector as mysql 
con = mysql.connect(host='localhost',database='teste',user='saulo',password='Olivetti@5!', auth_plugin='mysql_native_password')
cursor = con.cursor()

### extrai coluna valor do dictionary e grava arquivo .csv  
def ProcessaLinha(df_grava):
    df_montado = pd.DataFrame([df_grava])
    ##for linha in df_grava:
    valor = str(df_grava['value'])
    timestamp = str(df_grava['timestamp'])
    ###df_montado.columns = ['coluna1','coluna2','coluna3','coluna4','coluna5','coluna6','coluna7'] 
    ##df_montado.to_csv('/tmp/people.csv', mode='a', index=False, sep='|', header=False)
    comando = f'''insert into kafka values ("{valor}", "{timestamp}")'''
    print(comando)
    #cursor.execute(comando)     
    #con.commit()
    ##return df_montado
        
####        for message in consumer:
####    message = message.value
####    collection.insert_one(message)
####    print('{} added to {}'.format(message, collection))




In [ ]:
### inicializa instancia spark 
spark = pyspark.sql.SparkSession \
          .builder \
          .appName("APP") \
          .getOrCreate()

### inicia leitura streaming do inicio 
df = spark\
      .readStream \
      .format("kafka") \
      .option("kafka.bootstrap.servers", "localhost:9092") \
      .option("subscribe", "saulo-teste") \
      .option("startingOffsets",""" {"saulo-teste":{"0":0}}""") \
      .option("failOnDataLoss", "false") \
      .load()


###query = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)", "topic","CAST(timestamp AS STRING)", "CAST(offset AS STRING)", "CAST(partition AS STRING)" ) \
query = df.writeStream \
    .foreach(ProcessaLinha) \
    .start()

query.awaitTermination()

22/06/06 12:52:29 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-daee7130-d775-44c3-bdc9-a9b14b868e41. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/06/06 12:52:29 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
insert into kafka values ("bytearray(b'msg 340')", "2022-06-06 12:48:03.790000");
